Quete 3 scrapping chuck Norris

In [2]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [3]:
# scrapping des noms des fichiers
# url = "http://www.chucknorrisfacts.fr/facts/history"
# url = "http://www.chucknorrisfacts.fr/facts/top/1"
url = "https://www.allocine.fr/personne/fichepersonne-30367/filmographie/"

# contournement d'éventuels protection anti-scrapping
navigator = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

response = requests.get(url, headers={"User-Agent": navigator})
# response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Trouver tous les liens sur la page
liens = soup.find_all("a")


# Nous filtrons maintenant uniquement sur les films dont l'artiste est le réalisateur :
soup_director = soup.find('div', {"class": "gd-col-left"})
# Puis nous cherchons tous les liens contenus dans cet encadré :
links = soup_director.find_all('a')
print(f"\n :\n{links} \n")

In [5]:
links[0]

<a class="responsive-table-link blue-link" href="/film/fichefilm_gen_cfilm=296168.html" title="Oppenheimer">Oppenheimer</a>

In [6]:
print(links[0]["href"])
print(links[0]["title"])

/film/fichefilm_gen_cfilm=296168.html
Oppenheimer


Nous allons pouvoir les stocker dans un DataFrame :

In [7]:
df_url = pd.DataFrame({"title": [i["title"] for i in links],"href": [i["href"] for i in links]})
df_url.head(3)

,title,href
0,Oppenheimer,/film/fichefilm_gen_cfilm=296168.html
1,Tenet,/film/fichefilm_gen_cfilm=251315.html
2,Untitled Howard Hughes Biopic,/film/fichefilm_gen_cfilm=190437.html


Scraping de niveau 2 : date pour un premier film
Nous allons maintenant devoir collecter les informations de chacun des liens du DataFrame.
Nous nous apercevons que Allociné renseigne des liens "courts" : il n'y a pas le début de l'URL, nous devrons les compléter.

Essayons sur un seul film (le second : Tenet). Nous créons un script pour récupérer le contenu depuis un lien "court.

In [8]:
url = "https://www.allocine.fr" + "/film/fichefilm_gen_cfilm=251315.html"
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
detail = soup.find("div", {"class": "meta-body-item meta-body-info"})
detail

<div class="meta-body-item meta-body-info">
<span class="ACrL2ZACrpbG0vYWdlbmRhL3NlbS0yMDIwLTA4LTI2Lw== date blue-link">
26 août 2020
</span>
<strong>
en salle
</strong>
<span class="spacer">/</span>
2h 30min
<span class="spacer">/</span>
<span class="ACrL2ZACrpbG1zL2dlbnJlLTEzMDI1Lw==">Action</span>,
<span class="ACrL2ZACrpbG1zL2dlbnJlLTEzMDIxLw==">Science Fiction</span>
</div>

 la durée (2h 30min) n'est pas dans une balise spécifique. Nous allons passer par du RegEx pour l'extraire.

In [14]:
import re
re.findall(r"(\d+)(h )(\d+)(min)", str(detail))

[('2', 'h ', '30', 'min')]

Cela semble correct. Nous pouvons maintenant convertir en minutes la durée du film :

In [13]:
def crawl(url_short):
    url = "https://www.allocine.fr" + url_short
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    detail = soup.find("div", {"class": "meta-body-item meta-body-info"})
    regex_output = re.findall(r"(\d+)(h )(\d+)(min)", str(detail))[0]
    return int(regex_output[0]) * 60 + int(regex_output[2])

Utilisons cette fonction sur Tenet :

In [11]:
crawl("/film/fichefilm_gen_cfilm=251315.html")

150

Essayons sur le premier film :

In [15]:
crawl("/film/fichefilm_gen_cfilm=296168.html")

181

In [16]:
# modif foncrtion en intégrant un try except si le film n'est pas encore sortie
def crawl(url_short):
    url = "https://www.allocine.fr" + url_short
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    detail = soup.find("div", {"class": "meta-body-item meta-body-info"})
    try:
        regex_output = re.findall(r"(\d+)(h )(\d+)(min)", str(detail))[0]
        return int(regex_output[0]) * 60 + int(regex_output[2])
    except:
        pass



Application à l'ensemble des films de la liste
Il n'y a plus qu'à appliquer avec apply puis de calculer la moyenne :

In [18]:
df_url["duration_min"] = df_url["href"].apply(crawl)
duree_film = df_url["duration_min"].mean()


Les films de Christopher Nolan font en moyenne 137 minutes, soit plus de 2h. :
138.83 



In [24]:
heures=int(duree_film //60)
minutes=int(duree_film %60)
print(f"\nLes films de Christopher Nolan font en moyenne 139 minutes, soit :\n{heures}h{minutes}min \n")


Les films de Christopher Nolan font en moyenne 139 minutes, soit :
2h18min 



### Challenge

-Essaye de calculer l'âge moyen des acteurs et actrices dans le film Interstellar.

In [34]:
df_url.head()
# Résumé des informations du dataframe
your_dataframe = df_url
print(f"\nshape: {your_dataframe.shape} \nlist columns :\n{your_dataframe.columns.tolist()} ")
print(f"liste des colonnes numeriques: \n{your_dataframe.select_dtypes(include=[np.number]).columns.tolist()}\n")
print(f"liste des colonnes non numeriques: \n{your_dataframe.select_dtypes(exclude=[np.number]).columns.tolist()} ")
print(f"Noms des colonnes avec au moins une valeur NA : {your_dataframe.columns[your_dataframe.isna().any()].tolist()}")
print(f"Nombre de lignes avec au moins une valeur NA : {your_dataframe.isna().any(axis=1).sum()}")
print(f"Colonne avec des na :{your_dataframe.isna().sum()} \n")
print(f"\ndf head :\n{your_dataframe.head(2)} \n")
print(f"\ndf describe :\n{your_dataframe.describe()} \n")


shape: (26, 3) 
list columns :
['title', 'href', 'duration_min'] 
liste des colonnes numeriques: 
['duration_min']

liste des colonnes non numeriques: 
['title', 'href'] 
Noms des colonnes avec au moins une valeur NA : ['duration_min']
Nombre de lignes avec au moins une valeur NA : 3
Colonne avec des na :title           0
href            0
duration_min    3
dtype: int64 


df head :
         title                                   href  duration_min
0  Oppenheimer  /film/fichefilm_gen_cfilm=296168.html         181.0
1        Tenet  /film/fichefilm_gen_cfilm=251315.html         150.0 


df describe :
       duration_min
count     23.000000
mean     138.826087
std       59.495341
min        3.000000
25%      114.500000
50%      143.000000
75%      166.500000
max      242.000000 



In [59]:
nom_film = "interstellar"
nom_film = nom_film.capitalize()

url_film = my_dataframe["href"][my_dataframe["title"] == nom_film].values.item()
print(f"\nurls_interstellar :\n{url_film} \n")

url_casting = url_film.replace(".html", "").replace("_gen_cfilm=","-")+"/casting/"
print(f"\nurl_casting :\n{url_casting} \n")


urls_interstellar :
/film/fichefilm_gen_cfilm=114782.html 


url_casting :
/film/fichefilm-114782/casting/ 



In [48]:
duree_film = crawl(url_film)
heures = int(duree_film // 60)
minutes = int(duree_film % 60)
print(f"\nLe film interstellar dure {      duree_film} minutes, soit :\n{heures}h{minutes}min \nC'est supérieur à la moyenne generale de ses films qui est de 2h18")


Le film interstellar dure 169 minutes, soit :
2h49min 
C'est supérieur à la moyenne generale de ses films qui est de 2h18


### age moyen des acteurs du film interstellar

In [151]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

#  fonction recensant les liens du site allocine d'un producteur de films
def dataframe_liens_producteur(url):
    try:
        # Contournement d'éventuelles protections anti-scrapping
        navigateur = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

        # Faire une requête HTTP pour obtenir le contenu de la page
        reponse = requests.get(url, headers={"User-Agent": navigateur})

        # Parser le contenu HTML avec BeautifulSoup
        soupe = BeautifulSoup(reponse.text, "html.parser")
        # on ne prend que le body
        soupe = soupe.body
        # on supprime les balises scripts
        for script in soupe("script"):
            script.decompose()

        # Trouver tous les liens sur la page
        liens = soupe.find_all("a")

        # Filtrer uniquement sur les films dont l'artiste est le réalisateur
        soupe_directeur = soupe.find('div', {"class": "gd-col-left"})

        # Chercher tous les liens contenus dans cet encadré
        liens = soupe_directeur.find_all('a')

        # Créer un dataframe avec les titres et les liens
        df_url_producteur = pd.DataFrame(
            {"title": [i["title"] for i in liens], "href": [i["href"] for i in liens]})

        return df_url_producteur
    except Exception as e:
        print(f"Erreur dans dataframe_liens_producteur: {e}")
        return None
# ----------------------------------------------------------------
# Récupère les liens des acteurs (casting) à partir de l'URL du film
def obtenir_liens_acteurs(url_court):
    try:
        url = "https://www.allocine.fr" + url_court
        html = requests.get(url)
        soupe = BeautifulSoup(html.text, 'html.parser')
        # on ne prend que le body
        soupe = soupe.body
        # on supprime les balises scripts
        for script in soupe("script"):
            script.decompose()
        sections_acteurs = soupe.find_all(
            "div", {"class": "card person-card person-card-col"})
        liens_acteurs = []
        for acteur in sections_acteurs:
            a_element = acteur.find('a', {"class": "meta-title-link"})
            if a_element is not None:
                liens_acteurs.append(a_element['href'])
        return liens_acteurs
    except Exception as e:
        print(f"Erreur dans obtenir_liens_acteurs: {e}")
        return []
#----------------------------------------------------------------
# Récupère l'âge de l'acteur à partir de son lien
def obtenir_age_acteur(lien_acteur):
    try:
        url = "https://www.allocine.fr" + lien_acteur
        html = requests.get(url)
        soupe = BeautifulSoup(html.text, 'html.parser')
        # on ne prend que le body
        soupe = soupe.body
        # on supprime les balises scripts
        for script in soupe("script"):
            script.decompose()
        elements_meta = soupe.find_all("div", {"class": "meta-body-item"})
        # Regex "Age\s*" correspond à "Age" suivi de zéro ou plusieurs espaces
        element_age = soupe.find(
            "span", string=re.compile(r"Age\s*", re.IGNORECASE))
        if element_age:
            # Trouve le prochain élément frère qui est une balise <strong>
            age = element_age.find_next_sibling("strong")
            if age:
                return age.text
        return None
    except Exception as e:
        print(f"Erreur dans obtenir_age_acteur: {e}")
        return None
# ----------------------------------------------------------------
# Calcule l'âge moyen des acteurs à partir de leurs liens
def calculer_age_moyen(liens_acteurs):
    try:
        ages = []
        for lien in liens_acteurs:
            age = obtenir_age_acteur(lien)
            # Vérifie si l'âge est un nombre
            if age is not None and age.isdigit():
                ages.append(int(age))
        age_moyen = sum(ages) / len(ages) if ages else None
        age_moyen = round(age_moyen, 2)
        return age_moyen
    except Exception as e:
        print(f"Erreur dans calculer_age_moyen: {e}")
        return None
# ----------------------------------------------------------------
# Fonction principale pour obtenir l'âge moyen des acteurs d'un film d'un producteur au choix
def age_moyen_acteur(nom_film, my_dataframe):
    try:
        nom_film = nom_film.capitalize()
        # print(f"\nnom_film :\n{nom_film} \n")
        # Construit l'URL du film à partir du dataframe
        url_film = my_dataframe["href"][my_dataframe["title"] == nom_film].values.item().replace(".html", "").replace("_gen_cfilm=", "-")
        url_casting = url_film.replace(".html", "").replace("_gen_cfilm=", "-")+"/casting/"
        # print(f"\nurl_film :\n{url_film} \n")
        # print(f"\nurl_casting :\n{url_casting} \n")
        liens_acteurs = obtenir_liens_acteurs(url_casting)
        age_moyen = calculer_age_moyen(liens_acteurs)
        # print(f"\nliens_acteurs :\n{liens_acteurs} \n")
        # print(f"\nage_moyen :\n{age_moyen} \n")
        if age_moyen is not None:
            print(f"L'âge moyen des acteurs du film {nom_film} est de {age_moyen} ans.")
        else:
            print("Aucun âge trouvé pour les acteurs du film.")
    except Exception as e:
        print(f"Erreur dans age_moyen_acteur: {e}")

In [160]:
# Test fonction age moyen
# url du producteur Christopher Nolan
url = "https://www.allocine.fr/personne/fichepersonne-30367/filmographie/"
nom_film = 'interstellar'
my_dataframe1 = dataframe_liens_producteur(url)
# my_dataframe = df_url

age_moyen_acteur(nom_film, my_dataframe1)

L'âge moyen des acteurs du film Interstellar est de 53.11 ans.
